# 基于word2vec 方式，提取文本特征
* 训练word2vec 模型
* 基于word2vec 模型对文章提取特征

In [23]:
import sys
sys.path.append("/Users/zhengwenjie/AI/work/ML_3/2017-CCF-BDCI-AIJudge")
from collections import defaultdict
import pandas as pd
from config.db_config import Config
from utils import LOGGER
from gensim.models import Word2Vec
import numpy as np
config = Config()

In [2]:
config.data_csv_path

'/Users/zhengwenjie/AI/work/ML_3/2017-CCF-BDCI-AIJudge/data/output/corpus/data1.csv'

## 1. 训练word2vec 模型

In [7]:
def get_raw_documents():
    '''
    加载文章列表数据
    :return:  列表数据的list格式数据 ["xxx xxx xxx","abc dd bb"]
    '''
    df_all = pd.read_csv(config.data_csv_path, encoding='utf8')
    documents = df_all['content'].values
    LOGGER.log('documents number %d' % len(documents))
    return documents

def get_words_list(documents):
    '''
    获取word2vec 训练样本数据
    :param documents:  文章的list 格式:["word1 word2 word3", "word1 word2 word3"]
    :return: 格式： [["cat", "say", "meow"], ["dog", "say", "woof"]]
    '''
    texts = [[word for word in document.split(' ')] for document in documents]
    frequency = defaultdict(int)
    for text in texts:
        for token in text:
            frequency[token] += 1
    texts = [[token for token in text if frequency[token] >= 5] for text in texts]
    return texts

def train_word2vec(texts):
    '''
    :param texts:  iterable can be simply a list of lists of tokens
        格式： [["cat", "say", "meow"], ["dog", "say", "woof"]]
    :return:
    '''
    LOGGER.log('Train Model...')
    w2v = Word2Vec(texts, size=config.w2v_dim, window=5, iter=15, workers=12, seed=config.seed)
    w2v.save(config.model_w2v)
    LOGGER.log('Save done!')
    return w2v

In [8]:
raw_documents = get_raw_documents()
raw_documents[0:2]

2019-03-25 22:10:45 documents number 999


array(['公诉 机关 霍邱县 人民检察院 被告人 许某 1975 日生 2012 因涉嫌 危险 驾驶 霍邱县 公安局 取保候审 2013 日经 本院 取保候审 霍邱县 人民检察院 以霍检 刑诉 2013 起诉书 指控 被告人 许某 甲犯 危险 驾驶 2013 日向 本院 提起公诉 本院 依法 简易程序 实行 独任 审判 2013 公开 开庭审理 本案 霍邱县 人民检察院 检察员 胡涛 被告人 许某 到庭 参加 诉讼 现已 审理 终结 霍邱县 人民检察院 指控 2012 被告人 许某 酒后 驾驶 二轮 摩托车 沿霍寿路 由南向北 行驶 霍寿路 公园路 交叉口 路边 行人 相撞 公安民警 查获 六安市 疾病 预防 控制中心 鉴定 许某 血液 乙醇 含量 169.64 mg 100ml 上述事实 被告人 开庭审理 过程 无异议 被害人 杨正响 陈述 证人 李某 证言 六安市 疾病 预防 控制中心 检验 报告 六安市 疾控交 检字 2012 155 霍邱县 公安局 交通管理 大队 呼吸 酒精 检测 抽取 当事人 血样 登记表 驾驶 信息 查询 道路 交通事故 赔偿 调解 协议书 经济 赔偿 凭证 谅解 被告人 户籍 信息 证据 证明 足以认定',
       '公诉 机关 海口市 龙华区 人民检察院 被告人 王某 海口市 龙华区 人民检察院 海龙 检公 刑诉 774 起诉书 指控 被告人 王某 乙犯 贩卖毒品 日向 本院 提起公诉 本院 依法 简易程序 实行 独任 审判 公开 开庭审理 本案 海口市 龙华区 人民检察院 指派 检察员 余荣标 出庭 支持 公诉 被告人 王某 到庭 参加 诉讼 现已 审理 终结 公诉 机关 指控 时许 被告人 王某 海口市 龙华区 龙昆 北路 温泉 大酒店 门前 人民币 550 价格 购毒 人员 王某 贩卖 小包 毒品 鉴定 一小 包含 海洛因 成分 净重 0.9980 一小 包含 甲基苯丙胺 成分 净重 1.0469 交易 公安民警 抓获 公诉 机关 被告人 王某 无视 国家 法律 非法 贩卖毒品 海洛因 甲基苯丙胺 已触犯 中华人民共和国 刑法 第三 四十七 第四款 贩卖毒品 追究其 刑事责任 提请 本院 依法 判处 被告人 王某 起诉书 指控 事实 罪名 持异议 审理 查明 时许 事先 电话 被告人 王某 海口市 龙华区 龙昆 北

In [9]:
words_list = get_words_list(raw_documents)

In [10]:
w2v=train_word2vec(words_list)

2019-03-25 22:10:51 Train Model...
2019-03-25 22:10:54 Save done!


In [19]:
# 计算相近词
w2v.wv.most_similar("物品")

[('发还', 0.7579845190048218),
 ('抽水机', 0.7330999374389648),
 ('扣押', 0.7082433700561523),
 ('文件', 0.6904174089431763),
 ('领条', 0.6790323257446289),
 ('清单', 0.6725605726242065),
 ('笔记本电脑', 0.6546766757965088),
 ('搜查', 0.6536903381347656),
 ('退赃', 0.6419256329536438),
 ('电脑', 0.6370885372161865)]

In [20]:
# 词对应的向量表示
w2v.wv.word_vec("物品")

array([-0.6236638 , -0.61912614, -0.7919067 ,  0.48009562, -0.5801371 ,
       -0.29370427, -0.5955131 , -0.02454433, -0.24867605,  0.07943497,
       -0.5752363 , -0.617045  , -0.3207079 ,  0.89897263,  0.36863464,
        0.40289462,  0.6085367 ,  1.420481  ,  0.599323  ,  0.20803966,
       -0.12121307,  0.13925071,  1.2605767 ,  0.05564041, -0.44694483,
        1.0089743 ,  0.48730397,  0.06046977, -0.33408153,  0.0294657 ,
       -0.48045737,  0.5008066 , -0.51946855, -0.66699106, -0.14163353,
       -0.3861232 ,  0.41355523,  0.23014474,  0.23972002, -0.6871735 ,
       -0.00738693, -0.31829423,  0.7555705 , -0.09970266, -0.6014447 ,
       -0.1670253 ,  0.11512033,  0.36965522,  1.0360987 , -0.00690602,
       -0.02084903, -0.48849618, -0.9777356 , -0.35281476, -0.6302038 ,
       -0.5242214 , -0.8858683 , -0.1073685 ,  0.5759002 , -0.62995476,
        0.73343813, -0.47913954,  0.2190258 ,  0.6005578 ,  0.15623917,
       -0.72824204,  0.23931596, -0.72775173, -0.82250196, -0.62

# 2. 加载word2vec 模型，提取特征
针对文章每个记录提取特征（文章特征=文章分词后 每个词对应的特征向量，然后取平均数值）

In [21]:

def get_len(words_list):
    '''
    返回所有行数
    :param words_list:
    :return:
    '''
    rows=len(words_list)
    return rows


def build_w2v(texts):
    '''
    构建 文本的矩阵向量
    :param texts:
    :return:
    '''
    LOGGER.log('Start get w2v feat..')
    rows = get_len(texts)
    w2v_feat = np.zeros((rows, config.w2v_dim))
    w2v_feat_avg = np.zeros((rows, config.w2v_dim))
    i = 0
    for line in texts:
        num = 0
        for word in line:
            num += 1
            vec = model[word]
            w2v_feat[i, :] += vec
        w2v_feat_avg[i, :] = w2v_feat[i, :] / num  # 一个句子的向量表示＝所有词和的平均数值
        i += 1
        if i % 1000 == 0:
            LOGGER.log(i)

    feat_w2v_df = pd.DataFrame(w2v_feat)
    feat_w2v_avg_df = pd.DataFrame(w2v_feat_avg)
    LOGGER.log('Save w2v and w2v_avg feat done!')
    return feat_w2v_df, feat_w2v_avg_df

In [24]:
# 生成word2vec 格式的矩阵
words_list = get_words_list(get_raw_documents())
rows = get_len(words_list)
# 加载w2v 模型
model = Word2Vec.load(config.model_w2v)
# 针对样本数据构建w2v的特征
feat_w2v_df,feat_w2v_avg_df = build_w2v(words_list)

2019-03-25 22:52:27 documents number 999
2019-03-25 22:52:28 Start get w2v feat..


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


2019-03-25 22:52:30 Save w2v and w2v_avg feat done!


In [25]:
feat_w2v_df.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-54.250447,16.617582,-4.799216,-1.811491,5.626085,-94.880423,-56.224096,31.310888,-108.822245,27.189590,...,75.207152,10.958448,29.983353,-13.680185,-92.252895,-59.175050,0.960931,-28.740666,20.873822,21.861433
1,-18.831810,7.376106,44.599882,-5.539740,-12.247600,-109.822515,-118.148381,-3.235992,-108.672461,50.565711,...,70.378274,25.564536,77.129985,-17.968490,-91.718743,-111.672743,-0.371383,41.537117,26.575314,6.401734
2,-32.614559,-66.899649,48.596277,-32.145159,1.067856,-192.977929,-53.046814,111.261080,-130.131996,-4.020667,...,169.966411,-98.345126,128.344113,-13.004068,-127.812616,-12.804730,-2.833423,-80.896059,59.252219,12.181794
3,-7.872151,-3.096405,19.475321,-94.375564,-9.590727,-112.103800,-72.591482,-102.639545,-285.870639,-133.513115,...,197.345350,32.054230,79.103036,71.930918,-236.328585,16.059041,48.902316,-191.139000,48.552201,17.975337
4,1174.143111,-60.387402,-1041.529384,-861.222480,937.688801,-261.733481,-638.992887,-328.605206,-1219.450194,130.085578,...,-385.684864,1272.534124,-825.017688,-633.944896,-458.357564,-2253.509863,-462.191886,-450.968654,437.550783,-288.678710


In [26]:
feat_w2v_avg_df.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-0.420546,0.128818,-0.037203,-0.014043,0.043613,-0.735507,-0.435846,0.242720,-0.843583,0.210772,...,0.583001,0.084949,0.232429,-0.106048,-0.715139,-0.458721,0.007449,-0.222796,0.161813,0.169468
1,-0.097071,0.038021,0.229896,-0.028555,-0.063132,-0.566095,-0.609012,-0.016680,-0.560167,0.260648,...,0.362775,0.131776,0.397577,-0.092621,-0.472777,-0.575633,-0.001914,0.214109,0.136986,0.032999
2,-0.060397,-0.123888,0.089993,-0.059528,0.001978,-0.357367,-0.098235,0.206039,-0.240985,-0.007446,...,0.314753,-0.182121,0.237674,-0.024082,-0.236690,-0.023712,-0.005247,-0.149808,0.109726,0.022559
3,-0.011732,-0.004615,0.029024,-0.140649,-0.014293,-0.167070,-0.108184,-0.152965,-0.426037,-0.198976,...,0.294106,0.047771,0.117888,0.107200,-0.352204,0.023933,0.072880,-0.284857,0.072358,0.026789
4,0.349551,-0.017978,-0.310071,-0.256393,0.279157,-0.077920,-0.190233,-0.097828,-0.363040,0.038727,...,-0.114821,0.378843,-0.245614,-0.188730,-0.136457,-0.670887,-0.137598,-0.134257,0.130262,-0.085942
